In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data_df = pd.read_csv("cleaned_dataset.csv")
question_df = pd.read_csv("survey_question_list.csv")

In [ ]:
data_df = data_df.drop(columns=['G04Q11. How old are you?     ',
       'G02Q18. Please enter your citizenship:',
       'G04Q12. What is your highest completed level of education?',
       'G04Q13. What is your level of English proficiency?',
       'G04Q14. Do you have any previous experience in the field of machine learning?',
       'G04Q16. Do you have any previous experience with counterfactual explanation frameworks or causality frameworks?',
       'G02Q17. Do you have a medical background?',
       'G02Q08. From 1 (not at all) to 6 (perfectly), how well did you understand the metrics:',
       'G27Q249. '])
data_df

In [ ]:
def func(value):
  if(value<2):
    return "low"
  if(value<3):
    return "medium"
  return "high"

# Data processing for averaged experiment

In [ ]:
dataset = pd.DataFrame(columns=["prompt", "type", "answer"])
means = data_df.mean().drop(["id. Response ID"])

for i in range(240):
  #Scaling complexity separately
  if(i%8==7):
    dataset.loc[i] = [question_df.iloc[i//8].Question, i%8, func(means[i]*1.25+2.5)]
    continue
  dataset.loc[i] = [question_df.iloc[i//8].Question, i%8, func(means[i]-1)]

# Data processing for specific participants experiment

In [ ]:
# Row 131 = Participant A
# Row 27 = Participant B
# Row 177 = Participant C
# Row 169 = Participant C

dataset = pd.DataFrame(columns=["prompt","type","answer_100","answer_310","answer_356","answer_364"])

for i in range(240):
  if(i%8==7):
    dataset.loc[i] = [question_df.iloc[i//8].Question, i%8, func(data_df.drop(columns=["id. Response ID"]).iloc[27][i]*1.25+2.5),func(data_df.drop(columns=["id. Response ID"]).iloc[131][i]*1.25+2.5),func(data_df.drop(columns=["id. Response ID"]).iloc[169][i]*1.25+2.5), func(data_df.drop(columns=["id. Response ID"]).iloc[177][i]*1.25+2.5)]
    continue
  dataset.loc[i] = [question_df.iloc[i//8].Question, i%8, func(data_df.drop(columns=["id. Response ID"]).iloc[27][i]-1),func(data_df.drop(columns=["id. Response ID"]).iloc[131][i]-1),func(data_df.drop(columns=["id. Response ID"]).iloc[169][i]-1), func(data_df.drop(columns=["id. Response ID"]).iloc[177][i]-1)]

# Splitting data based on metrics

In [ ]:
# Function for finding a suitable split of data
def split(datas):
  for i in range(0,300000):
    if(i%10000==0):
      print("Epoch " + str(i))
    Tr0, Ts0 = train_test_split(datas, train_size=0.8,stratify=dataset.type, random_state=i)
    for j in range(8):
      answers = Ts0[Ts0.type==j].answer.values
      if(j==5):
        continue
      if not "low" in answers:
        break
      if not "medium" in answers:
        break
      if not "high" in answers:
        break
      if(j==7):
        print(i)
        return (Tr0,Ts0)


In [ ]:
# For reproducibility, this enables recreating the metric-wise split used in the experiments
Ts0 = dataset.iloc[[27,52,208,0,195,210,76,225,193,94,205,53,165,203,50,168,49,137,45,151,239,156,133,62,4,159,148,130,176,178,200,120,47,109,175,65,86,35,162,198,82,20,59, 206, 73, 95, 3, 126]].reset_index(drop=True)
Tr0 = dataset.drop([27,52,208,0,195,210,76,225,193,94,205,53,165,203,50,168,49,137,45,151,239,156,133,62,4,159,148,130,176,178,200,120,47,109,175,65,86,35,162,198,82,20,59, 206, 73, 95, 3, 126]).reset_index(drop=True)

Ts0.to_csv("testset_200_metric.csv")
Tr0.to_csv("trainset_200_metric.csv")

# Splitting data based on explanations

In [ ]:
testquestions = pd.concat([dataset.iloc[0:8],dataset.iloc[40:48], dataset.iloc[64:72], dataset.iloc[112:120], dataset.iloc[176:184], dataset.iloc[224:232]])
dropping_ixs = list(range(0,8)) + list(range(40,48))  + list(range(64,72)) + list(range(112,120)) + list(range(176,184)) + list(range(224,232))
Tr0 = dataset.drop(index=dropping_ixs).reset_index(drop=True)
Ts0 = testquestions.reset_index(drop=True)

Ts0.to_csv("testset_200_question.csv")
Tr0.to_csv("trainset_200_question.csv")